# Time Series Comparison

This notebook pulls in data from the NOAA CO-OPS NWLON stations and pulls in CORA data at those locations from the NOAA Open Data Dissemination (NODD) and plots water level time series for comparison.

## Configuration

Update these values as necessary

`station_id` is a list of NOAA ID's. One source of information is https://tidesandcurrents.noaa.gov/map/index.html?region=Texas

In [ ]:
# ═══════════════════════════════════════════════════════════════
#  CONFIGURATION - MODIFY THESE VALUES AS NEEDED
# ═══════════════════════════════════════════════════════════════

# NOAA ID's of stations to query
station_id = ['8665530', '8774770', '8771450', '8775237', '8775296', '8773037'] #, '8775870']


# Time period for data extraction
start_year = '2022'
start_month = '09'
start_day = '01'

end_year = '2022'
end_month = '11'
end_day = '01'


## Import python libraries.

In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask
import intake
# import xarray as xr
import scipy.spatial as sp
# import s3fs
# import geopy.distance
from scipy.spatial import KDTree
# import folium
# from folium import Marker
# from folium.plugins import HeatMap, MarkerCluster
# from branca.colormap import linear, LinearColormap
from tqdm import tqdm

### Define Utility functions
DO NOT MODIFY

In [ ]:
def area(x1, y1, x2, y2, x3, y3):

    return (abs((x1 * (y2 - y3) + x2 * (y3 - y1)
                + x3 * (y1 - y2)) / 2.0))

In [ ]:
def define_kd_tree(ds):
    e = ds.element.values.astype(int)
    emin1 = e-1
    num_elems = len(e)
    x_vals = ds.x.values
    y_vals = ds.y.values

    xe=np.mean(x_vals[emin1],axis=1)
    ye=np.mean(y_vals[emin1],axis=1)
    tree = sp.KDTree(np.c_[xe,ye])
    areas = [area(x_vals[emin1[k][0]],y_vals[emin1[k][0]],\
                  x_vals[emin1[k][1]],y_vals[emin1[k][1]],\
                  x_vals[emin1[k][2]],y_vals[emin1[k][2]])for k in range(0, num_elems)]
    return tree, areas, e, x_vals, y_vals

In [ ]:
def find_triangle(x_vals, y_vals, e,lat,lon):
    e = ds.element.values.astype(int)-1

    k = 10
    dist, ii = tree.query([lon,lat],k=k)
    ii = ii
    triangle_i = -1

    for i in range(0,k):

      a1 = area(lon,lat,\
                x_vals[e[ii[i]][0]],y_vals[e[ii[i]][0]],\
                x_vals[e[ii[i]][1]],y_vals[e[ii[i]][1]])

      a2 = area(lon,lat,\
                x_vals[e[ii[i]][1]],y_vals[e[ii[i]][1]],\
                x_vals[e[ii[i]][2]],y_vals[e[ii[i]][2]])
      a3 = area(lon,lat,\
                x_vals[e[ii[i]][0]],y_vals[e[ii][i][0]],\
                x_vals[e[ii[i]][2]],y_vals[e[ii][i][2]])

      t_area = a1 + a2 + a3
      if abs(t_area - areas[ii[i]]) < 0.00000001:
        triangle_i = ii[i]+1
        break
    if(triangle_i == -1):
        print("ERROR for " ,lat,lon)
    return triangle_i

## Get CORA dataset files
**Access the data on the NODD and initialize the available CORA datasets.** 

*This accesses a .yml file located on the NODD that shows which CORA output files are available to import.*

In [ ]:
# @title This accesses a .yml file located on the NODD that shows which CORA output files are available to import.
catalog = intake.open_catalog("s3://noaa-nos-cora-pds/CORA_V1.1_intake.yml",storage_options={'anon':True})
list(catalog)

**CORA-V1.1-fort.63: Hourly water levels <br>
CORA-V1.1-swan_DIR.63: Hourly mean wave direction <br>
CORA-V1.1-swan_TPS.63: Hourly peak wave periods <br>
CORA-V1.1-swan_HS.63: Hourly significant wave heights <br>
CORA-V1.1-Grid: Hourly water levels interpolated from model nodes to uniform 500-meter resolution grid <br>
All datasets denoted as timeseries are optimized for pulling long time series (greater than a few days) <br>
For up to a few days of data, use the regular dataset (not labeled timeseries)**

*Now, create an xarray dataset for the CORA data that you would like to use.*

In [ ]:
ds = catalog["CORA-V1.1-fort.63"].to_dask()

In [ ]:
# stationpointsfile = 'C:\\Users\\John.Ratcliff\\CORA\\HSOFS_GEC_Stations.csv'
# cora_stations = pd.read_csv(stationpointsfile)
# cora_stations['id'] = cora_stations['id'].astype(str)
# cora_stations.iloc[62]

## Get NWLON station data

**Create a dataframe of NWLON station ids and coordinates from the CO-OPS API where you want to do a comparison.**

In [ ]:
base_url = 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/.json'
params = {
    'type': 'waterlevels',
    'units': 'metric'
}
print(f'base_url: {base_url}, parameters: {params}')
response = requests.get(base_url, params=params)
content = response.json()

stations = content['stations']
stations_df = pd.DataFrame(stations)

# Include station name along with id, lat, lng
stations_df = stations_df[['id','name','lat','lng']]

# limit to the list of stations specified in the configuration section
stations_df = stations_df[stations_df['id'].isin(station_id)]

stations_df

**Loop through the station list to grab water level hourly heights for each station. Create a pandas dataframe of the time series data.**

In [ ]:

base_url = 'https://api.tidesandcurrents.noaa.gov/api/prod/datagetter'

for i in range(0,len(stations_df)):
    station_id = stations_df.id.iloc[i]
    print(f"Requesting data for station {station_id}")

    params = {
        'begin_date': f'{start_year}{start_month}{start_day}',
        'end_date': f'{end_year}{end_month}{end_day}',
        'station': station_id,
        'product': 'hourly_height',
        'datum': 'MSL',
        'time_zone': 'gmt',
        'units': 'metric',
        'format': 'json'
    }

    print(f'base_url: {base_url}, parameters: {params}')

    response = requests.get(base_url, params=params)
    content = response.json()

    # The API returns a list of dictionaries, each with scalar values.
    # We need to convert this structure into a list of lists or a list of dictionaries with list values
    # before creating a DataFrame.

    # Extracting data for 'time' and 'height'
    time_data = [d['t'] for d in content['data']]
    tnc_data = [d['v'] for d in content['data']]

    # Convert tnc_data to numeric using a list comprehension
    tnc_data = [float(x) for x in tnc_data]

    # Creating DataFrame if it's the first iteration
    if i == 0:
        df = pd.DataFrame({'time': time_data, 'tnc_' + stations_df.id.iloc[i]: tnc_data})
    # Appending 'height' data to existing DataFrame for subsequent iterations
    else:
        df['tnc_' + stations_df.id.iloc[i]] = tnc_data  # Append new height column
df.set_index('time', inplace=True)
df


## Calculate CORA water levels

### Get CORA node Coordinates

In [ ]:
tree, areas, e, x_vals, y_vals = define_kd_tree(ds)

In [ ]:
# Extract node coordinates from the mesh

node_coords = np.c_[ds.x.values, ds.y.values]

kdtree = KDTree(node_coords) # Build the kdtree that can be queried for nearest neighbor nodes to a geographic point

# Initialize
elem = np.zeros((len(stations_df), 1), dtype=int)
elem_nodes = np.zeros((len(stations_df), 3), dtype=int)
query_point = np.zeros((len(stations_df), 2))  # Assuming you want 2 columns for lat and lon
distances = np.zeros((len(stations_df), 3))    # Assuming you want 3 nearest neighbors
nearest = np.zeros((len(stations_df), 3), dtype=int)  # Assuming you want 3 nearest neighbors
dist = np.zeros((len(stations_df), 3), dtype=float)
weights = np.zeros((len(stations_df), 3), dtype=float)

# for i in range(0,len(stations_df)):
for i in tqdm(range(len(stations_df)), desc="Processing stations"):

    elem[i,:] = find_triangle(x_vals,y_vals,e,stations_df.lat.iloc[i],stations_df.lng.iloc[i])
    distances[i,:], nearest[i,:] = kdtree.query([stations_df.lng.iloc[i],stations_df.lat.iloc[i]], k=3)
    nearest[i,:] = nearest[i,:] + 1

    if elem[i]==-1:
        elem_nodes[i,:] = nearest[i,:]
    else:
        elem_nodes[i,:] = e[elem[i]-1]

    x_dist = stations_df.lng.iloc[i] - x_vals[elem_nodes[i,:]]
    y_dist = stations_df.lat.iloc[i] - y_vals[elem_nodes[i,:]]
    dist[i,:] = np.sqrt(x_dist * x_dist + y_dist * y_dist)

    if np.any(dist[i,:] ==0):
        weights[i,:] = np.where(dist[i,:] == 0, 1, 0)
    else:
        weights[i,:] = 1/dask.array.sqrt(x_dist * x_dist + y_dist * y_dist)

unique_nodes = np.unique(elem_nodes) # sorted unique nodes
mapped_triangle = np.searchsorted(unique_nodes, elem_nodes) # indices of the nodes


In [ ]:
unique_nodes[mapped_triangle]

In [ ]:
concat_nodes=np.concatenate(node_coords[elem_nodes-1], axis=0)
lat_nodes=concat_nodes[:,1]
lon_nodes=concat_nodes[:,0]

df_nodes=pd.DataFrame({'Lon': lon_nodes, 'Lat': lat_nodes})
df_nodes

In [ ]:
# disabled for now. It's not particularly useful

# # Create a base map centered on the first coordinate
# map_center = [stations_df['lat'].iloc[0], stations_df['lng'].iloc[0]]
# my_map = folium.Map(location=map_center, zoom_start=8)

# # Add markers for each coordinate
# for index, row in df_nodes.iterrows():
#     folium.CircleMarker(
#         location=[row['Lat'], row['Lon']],
#     ).add_to(my_map)

# for index, row in stations_df.iterrows():
#     folium.CircleMarker(
#         location=[row['lat'], row['lng']],
#         popup=row['id'], color='red'
#     ).add_to(my_map)

# tile = folium.TileLayer(
#         tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#         attr = 'Esri',
#         name = 'Esri Satellite',
#         overlay = False,
#         control = True
#        ).add_to(my_map)

# # Save the map to an HTML file
# # my_map.save("nodesmap.html")

# my_map

### Calculate CORA water levels

**Average the water levels at the 3 nodes of the element containing the station coordinates for the selected time period. Append these CORA values as new columns to the dataframe.**

In [ ]:
%%time

start_t = f'{start_year}-{start_month}-{start_day} 00:00:00'
end_t = f'{end_year}-{end_month}-{end_day} 23:00:00'
dt_range=pd.date_range(start_t, end_t, freq='h',inclusive='both')

num_ts = len(dt_range) # number of time samples
t = np.zeros((num_ts, 3), dtype=float) # preallocate with zeros
zeta_point = np.zeros((num_ts), dtype=float) # preallocate with zeros
mean_zeta = np.zeros((num_ts,len(stations_df)), dtype=float)

# for i in range(0,len(stations_df)):
for i in tqdm(range(len(stations_df)), desc="Processing stations"):
    zeta_tslice = ds["zeta"].sel(time=slice(start_t, end_t), node=unique_nodes[mapped_triangle[i]]-1).compute()
    t = zeta_tslice.values * weights[i]
    zeta_point = np.sum(t, axis=1) / np.sum(weights[i])
    mean_zeta_i = np.nanmean(zeta_tslice.values, axis=1)

    zeta_point[np.isnan(zeta_point)] = mean_zeta_i[np.isnan(zeta_point)]

    df['cora_'+stations_df.iloc[i,0]] = zeta_point


## Plot the data

**Plot the time series data for the NWLON observations and CORA for comparison.**

In [ ]:
ylabel = "MSL, m"
# num_plots = min(zeta_df.shape[1], len(stations_df))
for i in range(0,len(stations_df)):
  # Create title using both station ID and name
  station_id = stations_df.iloc[i, 0]  # id column
  station_name = stations_df.iloc[i, 1]  # name column
  title = f"Hourly Water Levels for {station_name}: {station_id}"

  # Create larger figure
  plt.figure(figsize=(15, 8))
  df[['tnc_'+stations_df.id.iloc[i],'cora_'+stations_df.iloc[i,0]]].plot(figsize=(15, 8))

  plt.title(title, fontsize=14, fontweight='bold')
  plt.xlabel('Date/Time (GMT)', fontsize=12)
  plt.ylabel(ylabel, fontsize=12)

  # Improve datetime label formatting
  plt.xticks(rotation=45, ha='right')

  # Add grid for better readability
  plt.grid(True, alpha=0.3)

  # Add legend with better positioning
  plt.legend(['TNC Observed', 'CORA Model'], loc='best')

  plt.tight_layout()
  plt.savefig(f"{station_id}_water_levels.png", dpi=300, bbox_inches='tight')
plt.show()